In [79]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

In [80]:
now = str(datetime.now() + timedelta(hours=9))[:-10].replace(" ", ":")
print(now)

# Credits

* [First R notebook](https://www.kaggle.com/kailex/m5-forecaster-v2)
* [Python translation](https://www.kaggle.com/kneroma/m5-forecast-v2-python)

# Changes
* v5 : try to optimise the LGBM params (go below in lgbm params section to see changes)
* v4 : add df, X_train deletion before training step --> increasing train sample without memeroy issues

In [81]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category",
            "event_type_1": "category", "event_type_2": "category",
            "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [82]:
pd.options.display.max_columns = 50

In [83]:
h = 28 
max_lags = 57
tr_last = 1941 # = datetime(2016, 5, 22)
fday = datetime(2016,5,23)
fday

datetime.datetime(2016, 5, 23, 0, 0)

In [84]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv("./sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv("./calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv("./sales_train_evaluation.csv", #"./sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [85]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [86]:
FIRST_DAY = 1 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [87]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

CPU times: user 16 s, sys: 2.13 s, total: 18.2 s
Wall time: 18.2 s


(46881677, 22)

In [88]:
#df.drop("Unnamed: 0", axis=1, inplace=True)

In [89]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_1,12.0,2011-01-29,11101,2,1,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
1,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_2,15.0,2011-01-30,11101,3,2,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
2,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_3,0.0,2011-01-31,11101,1,3,1,2011,0,0,0,0,0.0,0.0,0.0,0.46
3,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_4,0.0,2011-02-01,11101,5,4,2,2011,0,0,0,0,1.0,1.0,0.0,0.46
4,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_5,0.0,2011-02-02,11101,6,5,2,2011,0,0,0,0,1.0,0.0,1.0,0.46


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46881677 entries, 0 to 46881676
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
dtypes: datetime64[ns](1), float32(5), int16(14), object(2)
memory us

In [91]:
%%time

create_fea(df)
df.shape

CPU times: user 1min 39s, sys: 4.53 s, total: 1min 43s
Wall time: 1min 43s


(46881677, 31)

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46881677 entries, 0 to 46881676
Data columns (total 31 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
 22  lag_7         float32       
 23  lag_28        float32       
 

In [93]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_1,12.0,2011-01-29,11101,2,1,1,2011,0,0,0,0,0.0,0.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,4,1,29
1,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_2,15.0,2011-01-30,11101,3,2,1,2011,0,0,0,0,0.0,0.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,4,1,30
2,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_3,0.0,2011-01-31,11101,1,3,1,2011,0,0,0,0,0.0,0.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,5,1,31
3,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_4,0.0,2011-02-01,11101,5,4,2,2011,0,0,0,0,1.0,1.0,0.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,5,1,1
4,HOBBIES_1_008_CA_1_evaluation,7,0,0,0,0,d_5,0.0,2011-02-02,11101,6,5,2,2011,0,0,0,0,1.0,0.0,1.0,0.46,NaN,NaN,NaN,NaN,NaN,NaN,5,1,2


In [94]:
df.dropna(inplace = True)
df.shape

(45204727, 31)

In [95]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [96]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [97]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 9 s, sys: 500 ms, total: 9.5 s
Wall time: 9.5 s


In [98]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

22

In [99]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [100]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[20]	valid_0's rmse: 3.07797
[40]	valid_0's rmse: 2.5894
[60]	valid_0's rmse: 2.45971
[80]	valid_0's rmse: 2.42265
[100]	valid_0's rmse: 2.41187
[120]	valid_0's rmse: 2.4036
[140]	valid_0's rmse: 2.39656
[160]	valid_0's rmse: 2.38906
[180]	valid_0's rmse: 2.38088
[200]	valid_0's rmse: 2.37349
[220]	valid_0's rmse: 2.36778
[240]	valid_0's rmse: 2.36252
[260]	valid_0's rmse: 2.35689
[280]	valid_0's rmse: 2.35335
[300]	valid_0's rmse: 2.34893
[320]	valid_0's rmse: 2.34543
[340]	valid_0's rmse: 2.34181
[360]	valid_0's rmse: 2.33782
[380]	valid_0's rmse: 2.33482
[400]	valid_0's rmse: 2.33245
[420]	valid_0's rmse: 2.32949
[440]	valid_0's rmse: 2.32635
[460]	valid_0's rmse: 2.3234
[480]	valid_0's rmse: 2.32148
[500]	valid_0's rmse: 2.3188
[520]	valid_0's rmse: 2.31687
[540]	valid_0's rmse: 2.31403
[560]	valid_0's rmse: 2.31191
[580]	valid_0's rmse: 2.30889
[600]	valid_0's rmse: 2.3068
[620]	valid_0's rmse: 2.30488
[640]	valid_0's rmse: 2.30259
[660]	valid_0's rmse: 2.30115
[680]	valid_0's rms

In [101]:
m_lgb.save_model("model_{}.lgb".format(now))

In [102]:
fday

datetime.datetime(2016, 5, 23, 0, 0)

In [103]:
res = []
for tdelta in range(-28, 28):
    day = fday + timedelta(days=tdelta)
    res.append(day)

In [104]:
len(res)

56

In [105]:
print(len(res[:28]))
res[:28]

28


[datetime.datetime(2016, 4, 25, 0, 0),
 datetime.datetime(2016, 4, 26, 0, 0),
 datetime.datetime(2016, 4, 27, 0, 0),
 datetime.datetime(2016, 4, 28, 0, 0),
 datetime.datetime(2016, 4, 29, 0, 0),
 datetime.datetime(2016, 4, 30, 0, 0),
 datetime.datetime(2016, 5, 1, 0, 0),
 datetime.datetime(2016, 5, 2, 0, 0),
 datetime.datetime(2016, 5, 3, 0, 0),
 datetime.datetime(2016, 5, 4, 0, 0),
 datetime.datetime(2016, 5, 5, 0, 0),
 datetime.datetime(2016, 5, 6, 0, 0),
 datetime.datetime(2016, 5, 7, 0, 0),
 datetime.datetime(2016, 5, 8, 0, 0),
 datetime.datetime(2016, 5, 9, 0, 0),
 datetime.datetime(2016, 5, 10, 0, 0),
 datetime.datetime(2016, 5, 11, 0, 0),
 datetime.datetime(2016, 5, 12, 0, 0),
 datetime.datetime(2016, 5, 13, 0, 0),
 datetime.datetime(2016, 5, 14, 0, 0),
 datetime.datetime(2016, 5, 15, 0, 0),
 datetime.datetime(2016, 5, 16, 0, 0),
 datetime.datetime(2016, 5, 17, 0, 0),
 datetime.datetime(2016, 5, 18, 0, 0),
 datetime.datetime(2016, 5, 19, 0, 0),
 datetime.datetime(2016, 5, 20, 0,

In [106]:
print(len(res[28:]))
res[28:]

28


[datetime.datetime(2016, 5, 23, 0, 0),
 datetime.datetime(2016, 5, 24, 0, 0),
 datetime.datetime(2016, 5, 25, 0, 0),
 datetime.datetime(2016, 5, 26, 0, 0),
 datetime.datetime(2016, 5, 27, 0, 0),
 datetime.datetime(2016, 5, 28, 0, 0),
 datetime.datetime(2016, 5, 29, 0, 0),
 datetime.datetime(2016, 5, 30, 0, 0),
 datetime.datetime(2016, 5, 31, 0, 0),
 datetime.datetime(2016, 6, 1, 0, 0),
 datetime.datetime(2016, 6, 2, 0, 0),
 datetime.datetime(2016, 6, 3, 0, 0),
 datetime.datetime(2016, 6, 4, 0, 0),
 datetime.datetime(2016, 6, 5, 0, 0),
 datetime.datetime(2016, 6, 6, 0, 0),
 datetime.datetime(2016, 6, 7, 0, 0),
 datetime.datetime(2016, 6, 8, 0, 0),
 datetime.datetime(2016, 6, 9, 0, 0),
 datetime.datetime(2016, 6, 10, 0, 0),
 datetime.datetime(2016, 6, 11, 0, 0),
 datetime.datetime(2016, 6, 12, 0, 0),
 datetime.datetime(2016, 6, 13, 0, 0),
 datetime.datetime(2016, 6, 14, 0, 0),
 datetime.datetime(2016, 6, 15, 0, 0),
 datetime.datetime(2016, 6, 16, 0, 0),
 datetime.datetime(2016, 6, 17, 0,

In [107]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    #for tdelta in range(0, 28):
    for tdelta in range(-28, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    #te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
    te_sub = te.loc[te.date >= (fday - timedelta(days=28)), ["id", "sales"]].copy()
    #te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), "id"].str.replace("validation$", "evaluation")
    te_sub.loc[te.date < fday, "id"] = te_sub.loc[te.date < fday, "id"].str.replace("evaluation$", "validation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_eval_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


#sub2 = sub.copy()
#sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
#sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission_eval_{}.csv".format(now),index=False)

-28 2016-04-25 00:00:00
-27 2016-04-26 00:00:00
-26 2016-04-27 00:00:00
-25 2016-04-28 00:00:00
-24 2016-04-29 00:00:00
-23 2016-04-30 00:00:00
-22 2016-05-01 00:00:00
-21 2016-05-02 00:00:00
-20 2016-05-03 00:00:00
-19 2016-05-04 00:00:00
-18 2016-05-05 00:00:00
-17 2016-05-06 00:00:00
-16 2016-05-07 00:00:00
-15 2016-05-08 00:00:00
-14 2016-05-09 00:00:00
-13 2016-05-10 00:00:00
-12 2016-05-11 00:00:00
-11 2016-05-12 00:00:00
-10 2016-05-13 00:00:00
-9 2016-05-14 00:00:00
-8 2016-05-15 00:00:00
-7 2016-05-16 00:00:00
-6 2016-05-17 00:00:00
-5 2016-05-18 00:00:00
-4 2016-05-19 00:00:00
-3 2016-05-20 00:00:00
-2 2016-05-21 00:00:00
-1 2016-05-22 00:00:00
0 2016-05-23 00:00:00
1 2016-05-24 00:00:00
2 2016-05-25 00:00:00
3 2016-05-26 00:00:00
4 2016-05-27 00:00:00
5 2016-05-28 00:00:00
6 2016-05-29 00:00:00
7 2016-05-30 00:00:00
8 2016-05-31 00:00:00
9 2016-06-01 00:00:00
10 2016-06-02 00:00:00
11 2016-06-03 00:00:00
12 2016-06-04 00:00:00
13 2016-06-05 00:00:00
14 2016-06-06 00:00:00
15

In [108]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_evaluation,0.847765,0.802760,0.796325,0.802332,0.939134,1.170267,1.159257,0.879443,0.797091,0.906605,0.920301,1.137026,1.246445,1.174614,0.870868,0.870944,0.867869,0.872465,1.014604,1.187792,1.162220,0.838532,0.817813,0.826443,0.807812,0.952146,1.169014,1.064838
1,FOODS_1_001_CA_1_validation,0.303300,0.302955,0.293349,0.260854,1.170537,1.371554,1.381326,0.919707,0.909289,0.859199,0.844554,0.885989,1.171737,1.073636,0.884911,0.796685,0.797344,0.824346,0.922890,1.236602,1.145396,0.823461,0.767630,0.747128,0.727724,1.003935,1.215868,1.223448
2,FOODS_1_001_CA_2_evaluation,0.959008,0.925489,0.933848,0.932309,1.131762,1.674059,1.548805,1.274939,0.994991,1.071454,1.115640,1.374140,1.795699,1.689526,1.167043,1.135079,1.113332,1.156065,1.332382,1.880385,1.587055,1.068415,1.022665,1.041703,0.994813,1.189042,1.732631,1.547227
3,FOODS_1_001_CA_2_validation,0.140332,0.160046,0.165402,0.145651,1.376056,1.849196,1.927982,1.048154,1.007550,0.931950,0.897766,1.067970,1.645142,1.446113,1.081279,1.022316,1.025456,1.075687,1.276703,1.869427,1.800279,1.260155,1.187056,1.153370,1.186923,1.179194,1.739611,1.540270
4,FOODS_1_001_CA_3_evaluation,0.915093,0.886508,0.839445,0.862481,0.964353,1.273545,1.275114,1.135638,0.951836,1.014154,1.106059,1.193417,1.297290,1.316266,1.034800,0.990007,0.990921,1.011857,1.077600,1.445338,1.289724,0.980868,1.058749,1.005037,0.925967,0.994199,1.271967,1.321314
5,FOODS_1_001_CA_3_validation,0.231606,0.213378,0.193925,0.220110,1.334667,1.306005,1.163026,0.935605,0.938178,0.844841,0.835960,0.920323,1.177649,1.179032,0.956967,1.028275,0.973537,1.005874,1.071878,1.619792,1.464466,1.082252,1.054329,1.010689,1.041454,1.042975,1.275893,1.236639
6,FOODS_1_001_CA_4_evaluation,0.370976,0.381299,0.381802,0.407900,0.419539,0.487322,0.454813,0.364846,0.371322,0.448405,0.449583,0.511559,0.528690,0.497710,0.432942,0.435471,0.443933,0.449680,0.486162,0.565450,0.560594,0.446502,0.452194,0.464316,0.462610,0.476717,0.541381,0.464896
7,FOODS_1_001_CA_4_validation,0.258191,0.193345,0.188468,0.206144,0.416459,0.511891,0.588233,0.426995,0.434377,0.450742,0.444781,0.450556,0.485403,0.381616,0.411343,0.399562,0.444401,0.445180,0.520380,0.596031,0.576372,0.455456,0.431674,0.436957,0.439586,0.427084,0.491995,0.508611
8,FOODS_1_001_TX_1_evaluation,0.358288,0.371961,0.382362,0.387136,0.425779,0.528888,0.505695,0.398893,0.410779,0.514653,0.487866,0.589388,0.625728,0.603295,0.491832,0.480238,0.451348,0.486101,0.495343,0.595002,0.573290,0.474549,0.461626,0.474162,0.461819,0.485319,0.582858,0.521429
9,FOODS_1_001_TX_1_validation,0.156215,0.145427,0.141992,0.154683,0.185017,0.211653,0.280221,0.744878,0.684229,0.638913,0.631074,0.687728,0.679195,0.633059,0.569651,0.563172,0.737954,0.618218,0.708701,0.809144,0.848759,0.610608,0.470237,0.522078,0.478048,0.391012,0.490513,0.511363


In [109]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum(), sub["id"].str.contains("evaluation$").sum()

(60980, 30490, 30490)

In [110]:
sub.shape

(60980, 29)

In [111]:
#kaggle competitions submit -c m5-forecasting-accuracy -f submission_eval_2020-06-29:09:49.csv -m "20200629(1) Evaluation version of the origin"; 1.27115